TAREA: Captura una o varias imágenes con monedas no solapadas. Tras visualizar la imagen, identifica de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras ello, la tarea se resuelve mostrando por pantalla el número de monedas y la cantidad de dinero presentes en la imagen. No hay restricciones sobre utilizar medidas geométricas o de color. ¿Qué problemas han observado?

Nota: Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc. 

Extras: Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo.



In [27]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Carga la imagen de las monedas
img = cv2.imread('monedas2.jpg')

# Convertir a RGB para visualizar
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convertir a escala de grises
img_gris = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Aplicar un desenfoque para suavizar la imagen
img_gris = cv2.GaussianBlur(img_gris, (5, 5), 0)

# Umbralización (binarización)
_, img_th1 = cv2.threshold(img_gris, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# Encontrar contornos
contornos, _ = cv2.findContours(img_th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Definir diámetros conocidos de monedas en mm
monedas = {
    25.75: 2.0,    # 2 euros
    23.25: 1.0,    # 1 euro
    24.35: 0.5,    # 50 céntimos
    22.25: 0.2,    # 20 céntimos
    19.75: 0.1,    # 10 céntimos
    21.25: 0.05,   # 5 céntimos
    18.75: 0.02,   # 2 céntimos
    16.26: 0.01    # 1 céntimo
}

# Variables para almacenar posiciones y diámetros de las monedas detectadas
monedas_detectadas = []

# Recorrer los contornos y calcular el diámetro
for c in contornos:
    area = cv2.contourArea(c)
    if area > 10:
        (cx, cy), radio = cv2.minEnclosingCircle(c)
        diametro_px = 2 * radio
        monedas_detectadas.append(((int(cx), int(cy)), diametro_px))

# Variable para guardar el factor de conversión
factor_conversion = None

# Función para manejar clics del ratón
def seleccionar_moneda(event, x, y, flags, param):
    global factor_conversion
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # Verificar si el clic está cerca de alguna moneda detectada
        for (pos, diametro_px) in monedas_detectadas:
            cx, cy = pos
            radio = diametro_px / 2
            
            # Verificar si el clic está dentro del radio de una moneda
            if np.sqrt((x - cx) ** 2 + (y - cy) ** 2) < radio:
                # Asumimos que la moneda seleccionada es la referencia
                diametro_mm = 25.75  # Asumimos que el usuario selecciona la moneda de 2 euros
                factor_conversion = diametro_mm / diametro_px
                print(f"Moneda seleccionada en ({cx}, {cy}) con diámetro de {diametro_px:.2f} px.")
                print(f"Factor de conversión actualizado: {factor_conversion:.4f} mm/px")
                break

# Mostrar la imagen y registrar el callback para eventos de ratón
cv2.imshow('Selecciona una moneda de referencia', img)
cv2.setMouseCallback('Selecciona una moneda de referencia', seleccionar_moneda)
# Mostrar las monedas detectadas en la imagen original
for (pos, diametro_px) in monedas_detectadas:
    cx, cy = pos
    cv2.circle(img_rgb, (cx, cy), int(diametro_px / 2), (255, 0, 0), 2)  # Dibuja el contorno en azul

# Mostrar el resultado en la ventana
cv2.imshow('Monedas Detectadas', img_rgb)
# Esperar hasta que el usuario cierre la ventana
cv2.waitKey(0)
cv2.destroyAllWindows()

# Verificar si el usuario seleccionó una moneda
if factor_conversion is None:
    print("No se seleccionó ninguna moneda.")
    exit()

# Ahora que tenemos el factor de conversión, procesamos las monedas detectadas
total_monedas = 0
total_dinero = 0.0

# Volver a recorrer los contornos y calcular el valor de las monedas
for c in contornos:
    area = cv2.contourArea(c)
    
    if area > 10:
        (cx, cy), radio = cv2.minEnclosingCircle(c)
        diametro_px = 2 * radio
        
        # Convertimos el diámetro de píxeles a milímetros usando el factor de conversión
        diametro_mm = diametro_px * factor_conversion
        
        # Buscamos la denominación de la moneda según su diámetro
        moneda_valor = None
        for diametro_moneda, valor in monedas.items():
            if abs(diametro_mm - diametro_moneda) < 0.75:  # Tolerancia de 0.75 mm
                moneda_valor = valor
                break
        
        # Si encontramos una moneda válida, sumamos al total
        if moneda_valor is not None:
            total_monedas += 1
            total_dinero += moneda_valor
            print(f"Moneda detectada: {moneda_valor} euros, Diámetro: {diametro_mm:.2f} mm")

# Mostramos el resultado final
print(f"Total de monedas: {total_monedas}")
print(f"Cantidad total de dinero: {total_dinero:.2f} euros")


Moneda seleccionada en (409, 500) con diámetro de 209.66 px.
Factor de conversión actualizado: 0.1228 mm/px
Moneda detectada: 1.0 euros, Diámetro: 23.40 mm
Moneda detectada: 2.0 euros, Diámetro: 25.75 mm
Moneda detectada: 0.1 euros, Diámetro: 19.37 mm
Moneda detectada: 0.2 euros, Diámetro: 21.95 mm
Moneda detectada: 0.5 euros, Diámetro: 24.36 mm
Moneda detectada: 0.01 euros, Diámetro: 15.80 mm
Moneda detectada: 0.02 euros, Diámetro: 18.48 mm
Moneda detectada: 0.05 euros, Diámetro: 20.62 mm
Total de monedas: 8
Cantidad total de dinero: 3.88 euros


El mundo real es muy variado, las imágenes no siempre se capturan con unas condiciones de iluminación tan buenas o controladas. Ejemplo con aplicación de variantes de umbralizados ofrecidas por OpenCV.

In [ ]:
#Carga imagen directamente en grises
imgorig = cv2.imread('MPs.jpg', cv2.IMREAD_GRAYSCALE) 

img = cv2.GaussianBlur(imgorig,(5,5),0)

#Umbralizados
ret,imth1 = cv2.threshold(img,150,255,cv2.THRESH_BINARY_INV)
thotsu,imth2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
print('Umbral escogido ', thotsu)
imth3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
imth4 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)
 
titles = ['Original', 'Fijo','Otsu th='+str(int(thotsu)),
            'Adaptivo promedio', 'Adaptivo Gaussiano']
images = [img, imth1, imth2, 255 - imth3, 255 - imth4]
 
for i in range(5):
    plt.subplot(2,5,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i], fontsize=7)
    plt.xticks([]),plt.yticks([])

    #Obtiene únicamente los contornos externos
    if i>0:
        res,imth = cv2.threshold(images[i],120,255,cv2.THRESH_BINARY)
        contornos, hierarchy= cv2.findContours(imth, 
        cv2.RETR_EXTERNAL , 
        cv2.CHAIN_APPROX_SIMPLE)  
        img_cont = np.zeros(img.shape)
        cv2.drawContours(img_cont, contornos, -1, (255,255,255), -1)  
        plt.subplot(2,5,i+6),plt.imshow(img_cont,'gray')
        plt.xticks([]),plt.yticks([])
plt.show()

Clasificación de microplásticos

In [ ]:
#Cargamos tres subimágenes de cada uno de los tres tipos considerados (el alquitrán no es microplástico)
imgF = cv2.imread('FRA.png') 
imgP = cv2.imread('PEL.png') 
imgT = cv2.imread('TAR.png') 

#Mostramos
plt.subplot(131)
plt.axis("off")
plt.imshow(imgF) 
plt.title('Fragmentos')
plt.subplot(132)
plt.axis("off")
plt.imshow(imgP) 
plt.title('Pellet')
plt.subplot(133)
plt.axis("off")
plt.imshow(imgT) 
plt.title('Alquitrán')

El objetivo de la siguiente tarea, descrita más abajo, es desarrollar tu propio clasificador basado únicamente en heurísticas desde características geométricas y/o de apariencia, para distinguir en las imágenes completas, las partículas de cada tipo, debiendo mostrar la bondad del clasificador haciendo uso de métricas para ello. La siguiente celda obtiene varias métricas para un conjunto de datos imaginario (y con etiquetas aleatorias). Si bien las trataremos con más detalle en teoría, muestro un repertorio de ellas, dando más peso a la matriz de confusión. La ejecución de la celda requiere instalar el paquete scikit-learn.

¿Qué es una matriz de confusión?
Se utiliza para mostrar el comportamiento de un clasificador para las distintas clases conocidas, se relacionan las etiquetas de las muestras anotadas frente a las predichas por el clasificador. Se busca una matriz diagonal, pero la perfección es infrecuente.

El siguiente ejemplo, muestra el modo de obtener la matriz de confusión para un hipotético problema con cuatro clases, y valores de anotación (variable y) y predicción (variable y_pred) obtenidos de forma aleatoria.

In [ ]:

import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)


# Numero de muestras
n = 100  
nclases = 4

# A falta de clasificador y conjunto de datos, creamos anotaciones y predicciones de forma aleatoria
# Vector aleatorio con etiquetas anotadas
y = [random.randint(0, nclases - 1) for _ in range(n)]
print('Anotaciones ' , y)

# Vector aleatorio con etiquetas predichas por un supuesto clasificador
y_pred = [random.randint(0, nclases - 1) for _ in range(n)]
print('Predicciones ' , y_pred)

print('¿Cómo de bien encajan anotación y predicción?')

#Cálculo de métricas
accuracy = accuracy_score(y, y_pred)
#Para más de una clase se define la forma de promediar
precision = precision_score(y, y_pred,average='weighted')
recall = recall_score(y, y_pred,average='weighted')
f1score = f1_score(y, y_pred,average='weighted')

print(f"Accuracy (TP/(n))= {accuracy}")
print(f"Precision (TP/(TP+FP)) = {precision}")
print(f"Recall (TP/(TP+FN)) = {recall}")
print(f"F1 Score (2*(precision*recall)/(precision+recall)) = {f1score}")


conf_matrix = confusion_matrix(y, y_pred)
plt.figure(figsize=(8,8))
sns.set(font_scale = 1.75)#tamaños tipografía
sns.set(font_scale = 3.0)

ax = sns.heatmap(
        conf_matrix, # confusion matrix 2D array 
        annot=True, # Muestra números en las celdas
        fmt='d', # valores enteros
        cbar=False, # sin barra de colores
        cmap='flag', # mapa de colores
        #vmax=175 # contraste de color
    )

#Etiquetas matriz de confusión
label_font = {'size':'25'}
ax.set_xlabel("Predicha", labelpad=-0.75, fontdict=label_font)
ax.set_ylabel("Real/Anotado", labelpad=20, fontdict=label_font)

TAREA: Las tres imágenes cargadas en la celda inicial, han sido extraidas de las imágenes de mayor tamaño presentes en la carpeta. La tarea consiste en extraer características (geométricas y/o visuales) e identificar patrones que permitan distinguir las partículas de cada una de las tres clases, evaluando los aciertos y fallos con las imágenes completas considerando las métricas mostradas y la matriz de confusión. La matriz de confusión, muestra para cada clase el número de muestras que se clasifican correctamente de dicha clase, y el número de muestras que se clasifican incorrectamente por cada una de las otras dos clases.

En el trabajo [SMACC: A System for Microplastics Automatic Counting and Classification](https://doi.org/10.1109/ACCESS.2020.2970498), las características geométricas utilizadas fueron:

- Área en píxeles
- Perímetro en píxeles
- Compacidad (relación entre el cuadrado del perímetro y el área de la partícula)
- Relación del área de la partícula con la del contenedor
- Relación del ancho y el alto del contenedor
- Relación entre los ejes de la elipse ajustada
- Definido el centroide, relación entre las distancias menor y mayor al contorno

Si no se quedan satisfechos con la segmentación obtenida, es el mundo real, también en el README comento técnicas recientes de segmentación, que podrían despertar su curiosidad.